In [58]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [59]:
dat = pd.read_csv("./data/processed_data.csv")
vax = pd.read_csv("./data/processed_vax.csv")
sym = pd.read_csv("./data/processed_symptoms.csv")

In [60]:
temp = pd.DataFrame(vax.groupby("vaersId").apply(lambda x: x.values.tolist()))
temp[0] = temp[0].apply(lambda x: [y[1:] for y in x][:2])
temp = pd.DataFrame(temp[0].tolist(), index=temp.index)
temp = temp.fillna("none")
temp[1] = temp[1].apply(lambda x: [] if x == "none" else x)

temp1 = pd.DataFrame(temp[0].tolist(), index=temp.index, columns=["vaxType1", "vaxManu1", "vaxRoute1", "vaxSite1", "covid1"])
temp2 = pd.DataFrame(temp[1].tolist(), index=temp.index, columns=["vaxType2", "vaxManu2", "vaxRoute2", "vaxSite2", "covid2"])
temp2 = temp2.fillna("UNK")

temp = temp1.reset_index().merge(temp2.reset_index(), on="vaersId")

In [61]:
def covid_actual(row, covid1, covid2):
    if (row[covid1] == 1) and (row[covid2] == "UNK"):
        return 1
    elif (row[covid1] == 0) and (row[covid2] == "UNK"):
        return 0
    else:
        return row[covid1] or row[covid2]

In [62]:
temp["covid"] = temp.apply(covid_actual, axis=1, covid1="covid1", covid2="covid2")

In [63]:
temp = temp.drop(["covid1", "covid2"], axis=1)

In [66]:
common_vaers = list(set(
    temp["vaersId"].tolist()).intersection(set(
        sym["vaersId"].tolist())).intersection(set(
            dat["vaersId"].tolist())))

In [67]:
temp = temp[temp["vaersId"].isin(common_vaers)].reset_index().drop("index", axis=1).copy()
sym = sym[sym["vaersId"].isin(common_vaers)].reset_index().drop("index", axis=1).copy()
dat = dat[dat["vaersId"].isin(common_vaers)].reset_index().drop("index", axis=1).copy()

In [72]:
sigma = dat.merge(sym, on="vaersId").merge(temp, on="vaersId")

In [75]:
sigma.to_csv("./data/combined.csv", index=False)